In [1]:
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import sys
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
from datetime import datetime
import copy
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [2]:
import platform; print(platform.platform())

macOS-14.6-arm64-arm-64bit


In [93]:
# df = pd.read_csv('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/labelled_data_500_30bps.csv')
# df = pd.read_csv('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/labelled_data_500.csv')
df = pd.read_csv('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/labelled_data_500_5bps.csv')

In [94]:
x = pd.to_datetime(df['date']).value_counts()
x.sort_index().index

DatetimeIndex(['2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04',
               '2021-06-07', '2021-06-08', '2021-06-09', '2021-06-10',
               '2021-06-11', '2021-06-15', '2021-06-16', '2021-06-17',
               '2021-06-18', '2021-06-21', '2021-06-22'],
              dtype='datetime64[ns]', name='date', freq=None)

In [95]:
train_set = df[pd.to_datetime(df['date']) <= pd.to_datetime('2021-06-07')]
test_data = df[pd.to_datetime(df['date']) == pd.to_datetime('2021-06-08')]
train_set.head()

,date,SP1,SV1,BP1,BV1,SP2,SV2,BP2,BV2,SP3,...,BV3,SP4,SV4,BP4,BV4,SP5,SV5,BP5,BV5,label
0,2021-06-01,10200.0,3294,10150.0,61,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,2
1,2021-06-01,10200.0,3294,10150.0,59,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,2
2,2021-06-01,10200.0,3324,10150.0,59,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,2
3,2021-06-01,10200.0,3324,10150.0,43,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,2
4,2021-06-01,10200.0,3324,10150.0,43,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,687,2


In [96]:
train_data =  train_set.iloc[:int(np.floor(train_set.shape[0] * 0.8)),:]
eval_data = train_set.iloc[int(np.floor(train_set.shape[0] * 0.8)):,:]

In [97]:
def normalize_data(df,cols,norm):
    #Normalizing using z-score
    if norm=='Z':
        scaler = StandardScaler()
        scaler.fit(df[cols])
        data = scaler.transform(df[cols])
    
    
    #Normalizing using DecPrec
    if norm=='DecPrec':
        k_len = np.ceil(np.log10(df[cols].abs().max()))
        # print(k_len)
        data = df[cols]/(10**k_len)

    return data,scaler

In [98]:
train_input , scaler = normalize_data(train_data.iloc[:,1:-1],train_data.iloc[:,1:-1].columns,'Z')
train_label = train_data.iloc[:,-1].to_numpy()
eval_input = scaler.transform(eval_data.iloc[:,1:-1])
eval_label = eval_data.iloc[:,-1].to_numpy()
test_input = scaler.transform(test_data.iloc[:,1:-1])
test_label = test_data.iloc[:,-1].to_numpy()

In [99]:
del [train_data,eval_data,df,train_set]

In [100]:
s=0
cc=0
key=0
obj=0
for key,obj in locals().items():
    cc= sys.getsizeof(obj)/1000000
    s+=cc
    if cc>20:
        print(key,cc)
print(s,'mb')

test_data 66.409622
82.29984699999997 mb


In [101]:
np.isnan(eval_input).any()

False

In [102]:
class Dataset_LOB(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, x,y, num_classes, T):
        """Initialization""" 
        # self.k = k
        self.num_classes = num_classes
        self.T = T
            
        # x = prepare_x(data)
        # y = get_label(data)
        x, y = data_classification(x, y, self.T)
        # y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [103]:
def data_classification(X, Y, T):
    [N, D] = X.shape
    # print(X.shape,T,N,Y.shape)
    df = np.array(X)
    # print(df.shape)
    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY


In [104]:
batch_size = 64
# dataset_train = Dataset_LOB(train_input,train_label, num_classes=3, T=100)
# dataset_eval = Dataset_LOB(eval_input,eval_label, num_classes=3, T=100)
dataset_test = Dataset_LOB(test_input,test_label, num_classes=3, T=100)

# test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

In [105]:
# train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
# val_loader = torch.utils.data.DataLoader(dataset=dataset_eval, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)
print(dataset_test.x.shape, dataset_test.y.shape)

torch.Size([282495, 1, 100, 20]) torch.Size([282495])


In [106]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[ 0.3677, -0.2225,  0.3676,  ..., -0.7481,  0.3625,  0.7571],
          [ 0.3677, -0.2225,  0.3676,  ..., -0.7481,  0.3625,  0.7555],
          [ 0.3677, -0.2228,  0.3676,  ..., -0.7481,  0.3625,  0.7555],
          ...,
          [ 0.3677, -0.2577,  0.3676,  ..., -0.7457,  0.3625,  0.7535],
          [ 0.3677, -0.2577,  0.3676,  ..., -0.7457,  0.3625,  0.7535],
          [ 0.3677, -0.2577,  0.3676,  ..., -0.7453,  0.3625,  0.7535]]]],
       dtype=torch.float64)
tensor([1])
torch.Size([1, 1, 100, 20]) torch.Size([1])


In [107]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

device = mps_device

tensor([1.], device='mps:0')


In [108]:
class deeplob(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,5)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        
        # lstm layers
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        # h0: (number of hidden layers, batch size, hidden size)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)
    
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)  
        
        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
#         x = torch.transpose(x, 1, 2)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        forecast_y = torch.softmax(x, dim=1)
        
        return forecast_y

In [109]:
model = deeplob(y_len = 3)
model.to(mps_device)
# device = mps_device

deeplob(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): Tanh()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): Tanh()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride

In [110]:
# model.load_state_dict(torch.load('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/fast-alphas/best_val_roLOB_30bps.pt', map_location=mps_device))

# model.load_state_dict(torch.load('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/fast-alphas/best_val_roLOB.pt', map_location=mps_device))
model.load_state_dict(torch.load('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/fast-alphas/best_val_roLOB_2603_5bps.pt', map_location=mps_device))


/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_60380/1946624102.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/Users/jand

<All keys matched successfully>

In [111]:

n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    model.eval()
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

Test acc: 0.5460


In [112]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.5459636453742545
              precision    recall  f1-score   support

           0     0.2184    0.4208    0.2875     30509
           1     0.8248    0.5816    0.6822    224891
           2     0.1627    0.3910    0.2298     27095

    accuracy                         0.5460    282495
   macro avg     0.4019    0.4645    0.3998    282495
weighted avg     0.6958    0.5460    0.5962    282495



In [113]:
all_predictions

array([1, 1, 1, ..., 2, 2, 2])

In [114]:
signal = (all_predictions-1)*-1

In [115]:
def getPos_Passive(signal):
    positions = np.zeros(signal.shape[0]+1)
    for i in range(0,len(signal)):
        if signal[i]!=0 and signal[i]!= positions[i]:
            positions[i+1] = signal[i]
        else:
            positions[i+1] = positions[i]
    return positions[1:]
    

    

In [116]:
positions_passive = getPos_Passive((all_predictions-1)*-1)

In [117]:
def getPos_Aggresive(signal):
    positions = np.zeros(signal.shape[0])
    positions[0] = signal[0]
    for i in range(1,len(signal)):
        if signal[i]!=0 and signal[i]!= signal[i-1]:
            # print(signal[i],positions[i-1],i,np.sign(signal[i]) == np.sign(positions[i-1]))
            if np.sign(signal[i]) == np.sign(positions[i-1]):
                positions[i] = positions[i-1]+signal[i]

            else:
                positions[i] = signal[i]

        else:
            positions[i] = positions[i-1]
            
    return positions
    

In [118]:
positions_aggesive = getPos_Aggresive((all_predictions-1)*-1)

In [119]:
max(positions_aggesive)

11.0

In [120]:
def calculatePnL_Mid(positions,df,signal):
    df['signal'] = signal
    df['positions'] = positions
    df['entryPrice'] = None
    df['exitPrice'] = None
    # df['Tick_Ret'] = None
    df['midPrice'] = (df['SP1']+df['BP1'])/2
    # df['entryMid'] = None

    df['isTrade'] = None
    df.loc[df['positions']!=df['positions'].shift(1),'isTrade'] = 1
    # df.loc[df['positions'].shift(1) > 0 ,'Tick_Ret'] = ((df.loc[df['positions'].shift(1) > 0, 'BP1'] - df['BP1'].shift(1)) / df['BP1'].shift(1))*df['positions'].shift(1)
    # df.loc[df['positions'].shift(1) < 0 ,'Tick_Ret'] = ((df.loc[df['positions'].shift(1) < 0, 'SP1'] - df['SP1'].shift(1)) / df['SP1'].shift(1))*df['positions'].shift(1)

    df.loc[(df['isTrade']==1) & (df['positions']>0),'entryPrice'] = df.loc[(df['isTrade']==1) & (df['positions']>0),'midPrice']
    df.loc[(df['isTrade']==1) & (df['positions']<0),'entryPrice'] = df.loc[(df['isTrade']==1) & (df['positions']<0),'midPrice']
    df.loc[(df['isTrade']==1) & (df['positions'].shift(1)>0),'exitPrice'] = df.loc[(df['isTrade']==1) & (df['positions'].shift(1)>0),'midPrice']
    df.loc[(df['isTrade']==1) & (df['positions'].shift(1)<0),'exitPrice'] = df.loc[(df['isTrade']==1) & (df['positions'].shift(1)<0),'midPrice']

    # df.loc[(df['isTrade']==1),'entryMid'] = df.loc[(df['isTrade']==1),'Mid']
    
    df['entryPrice'] = df['entryPrice'].ffill()
    df['TradeRet'] = np.where(df['isTrade'] == 1, ((df['exitPrice'] - df['entryPrice'].shift(1))/(df['entryPrice'].shift(1)))*df['positions'].shift(1), 0)
    df['cumRet'] = (1 + df['TradeRet']).cumprod()
    

In [121]:
def calculatePnL_BidAsk(positions,df,signal):
    df['signal'] = signal
    df['positions'] = positions
    df['entryPrice'] = None
    df['exitPrice'] = None
    # df['Tick_Ret'] = None
    df['midPrice'] = (df['SP1']+df['BP1'])/2
    # df['midPrice'] = (df['SP1']+df['BP1'])/2
    # df['entryMid'] = None

    df['isTrade'] = None
    df.loc[df['positions']!=df['positions'].shift(1),'isTrade'] = 1
    # df.loc[df['positions'].shift(1) > 0 ,'Tick_Ret'] = ((df.loc[df['positions'].shift(1) > 0, 'BP1'] - df['BP1'].shift(1)) / df['BP1'].shift(1))*df['positions'].shift(1)
    # df.loc[df['positions'].shift(1) < 0 ,'Tick_Ret'] = ((df.loc[df['positions'].shift(1) < 0, 'SP1'] - df['SP1'].shift(1)) / df['SP1'].shift(1))*df['positions'].shift(1)

    df.loc[(df['isTrade']==1) & (df['positions']>0),'entryPrice'] = df.loc[(df['isTrade']==1) & (df['positions']>0),'SP1']
    df.loc[(df['isTrade']==1) & (df['positions']<0),'entryPrice'] = df.loc[(df['isTrade']==1) & (df['positions']<0),'BP1']
    df.loc[(df['isTrade']==1) & (df['positions'].shift(1)>0),'exitPrice'] = df.loc[(df['isTrade']==1) & (df['positions'].shift(1)>0),'BP1']
    df.loc[(df['isTrade']==1) & (df['positions'].shift(1)<0),'exitPrice'] = df.loc[(df['isTrade']==1) & (df['positions'].shift(1)<0),'SP1']

    # df.loc[(df['isTrade']==1),'entryMid'] = df.loc[(df['isTrade']==1),'Mid']
    
    df['entryPrice'] = df['entryPrice'].ffill()
    df['TradeRet'] = np.where(df['isTrade'] == 1, ((df['exitPrice'] - df['entryPrice'].shift(1))/(df['entryPrice'].shift(1)))*df['positions'].shift(1), 0)
    df['cumRet'] = (1 + df['TradeRet']).cumprod()
    
    
       

In [122]:
# df = test_data.reset_index().iloc[99:][['SP1','BP1']]
# calculatePnL_BidAsk(positions_passive,df,signal)
# df.to_excel('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/pnl_verification.xlsx')

In [123]:
df = test_data.reset_index().iloc[99:][['SP1','BP1']]
calculatePnL_Mid(positions_passive,df,signal)
df.to_excel('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/pnl_verification_mid_5bps.xlsx')

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_60380/1016046938.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['entryPrice'] = df['entryPrice'].ffill()


In [124]:
# df = test_data.reset_index().iloc[99:][['SP1','BP1']]
# calculatePnL_BidAsk(positions_aggesive,df,signal)
# df.to_excel('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/pnl_verification_aggresive.xlsx')

In [125]:
df = test_data.reset_index().iloc[99:][['SP1','BP1']]
calculatePnL_Mid(positions_aggesive,df,signal)
df.to_excel('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/pnl_verification_mid_aggresive_5bps.xlsx')

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_60380/1016046938.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['entryPrice'] = df['entryPrice'].ffill()
